Importing the dataset

In [27]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Deep Learning/train.csv.zip')
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


Dropping the Nan values

In [28]:
data = data.dropna()

Defining the dependent and independent variables

In [29]:
X = data.drop('label', axis=1)
y = data['label']

Checking the shape of X, y

In [30]:
print(X.shape)
print(y.shape)

(18285, 4)
(18285,)


Importing the tensorflow libraries

In [31]:
import tensorflow as tf
print(tf.__version__)

2.8.2


Importing the tensorflow libraries

In [32]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM 
from tensorflow.keras.layers import Dense

Defining the vocabulary size

In [33]:
vocabsize = 5000

One Hot Representation

In [34]:
messages = X.copy()

In [35]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

Resetting the indexes

In [36]:
messages.reset_index(inplace=True)

Import nltk libraries

In [37]:
import nltk 
import re
from nltk.corpus import stopwords

In [38]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Data Preprocessing

In [39]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

Checking the corpus

In [40]:
corpus[:20]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

One Hot Representation

In [41]:
onehotrep = [one_hot(words, vocabsize) for words in corpus]
onehotrep[:5]

[[774, 4007, 255, 1656, 45, 4711, 4081, 1723, 939, 4229],
 [1385, 3055, 4312, 888, 4843, 594, 3874],
 [823, 2957, 1076, 4847],
 [2254, 1933, 1153, 3175, 1021, 2348],
 [553, 4843, 869, 3437, 283, 2266, 4843, 4599, 1614, 224]]

Embedding Representation

In [42]:
sentencelength = 20
embeddeddocs = pad_sequences(onehotrep, padding='pre', maxlen=sentencelength)
print(embeddeddocs)

[[   0    0    0 ... 1723  939 4229]
 [   0    0    0 ... 4843  594 3874]
 [   0    0    0 ... 2957 1076 4847]
 ...
 [   0    0    0 ... 1080 4236 1177]
 [   0    0    0 ... 2504 3791 4241]
 [   0    0    0 ...  298 4700  435]]


In [43]:
embeddeddocs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  774,
       4007,  255, 1656,   45, 4711, 4081, 1723,  939, 4229], dtype=int32)

Creating models

In [44]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocabsize, embedding_vector_features, input_length=sentencelength))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [45]:
len(embeddeddocs), y.shape

(18285, (18285,))

Importing libraries

In [46]:
import numpy as np
Xfinal = np.array(embeddeddocs)
yfinal = np.array(y)

Train test split

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xfinal, yfinal, test_size=0.33, random_state=42)

Training the model

In [48]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 35ms/step - loss: 0.3380 - accuracy: 0.8353 - val_loss: 0.2006 - val_accuracy: 0.9148
Epoch 2/10
192/192 [==============================] - 6s 31ms/step - loss: 0.1443 - accuracy: 0.9424 - val_loss: 0.1980 - val_accuracy: 0.9195
Epoch 3/10
192/192 [==============================] - 6s 33ms/step - loss: 0.1014 - accuracy: 0.9623 - val_loss: 0.2577 - val_accuracy: 0.9170
Epoch 4/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0716 - accuracy: 0.9736 - val_loss: 0.2807 - val_accuracy: 0.9122
Epoch 5/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0479 - accuracy: 0.9825 - val_loss: 0.3312 - val_accuracy: 0.9140
Epoch 6/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0316 - accuracy: 0.9899 - val_loss: 0.4082 - val_accuracy: 0.9175
Epoch 7/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0171 - accuracy: 0.9946 - val_loss: 0.4537 - val_accuracy: 0.914

Adding Dropout

In [49]:
# from tensorflow.keras.layers import Dropout

# embedding_vector_features=40
# model = Sequential()
# model.add(Embedding(vocabsize, embedding_vector_features, input_length=sentencelength))
# model.add(Dropout(0.3))
# model.add(LSTM(100))
# model.add(Dropout(0.3))
# model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuarcy'])

Performance metrics and Accuracy

In [50]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

Confusion Matrix

In [51]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[3137,  282],
       [ 253, 2363]])

Accuracy Score

In [52]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9113504556752279